## (실습) 항공 데이터 - 국가 코드 매핑하기
## 과제 : 용량 큰 데이터 처리 및 데이터 조인

In [ ]:
import pandas as pd
#pd.set_option('chained',None)

In [ ]:
asia = pd.read_csv('./data/dataset_raw_data.csv', encoding="cp949")

In [ ]:
asia

In [ ]:
asia.shape

In [ ]:
asia.columns

In [ ]:
asia.head()

In [ ]:
country_code = pd.read_csv('./data/country_code.csv',encoding="cp949")
country_code.head()

In [ ]:
country_code.columns

In [ ]:
country_code.tail()

In [ ]:
airport_code = pd.read_csv('./data/airport_code.csv',encoding="cp949")
airport_code

In [ ]:
airport_code.head()

In [ ]:
airport_code.head(1)#.iterrows()

In [ ]:
for a, row_a in airport_code.iterrows() :
    print(a, row_a)
    print("==="*5)

In [ ]:
for j, row_c in country_code.iterrows() :
    print(j, row_c)
    print("==="*5)

In [ ]:
import time

def measure_run_time(code_block):
    start_time = time.time()
    exec(code_block)
    end_time = time.time()
    execution_time = end_time - start_time
    print("실행 시간:", execution_time, "초")

In [ ]:
### 방법2 : 일반적 for문으로 작성 (시간비교)
measure_run_time("""
for a in range(len(airport_code)):
    row_a = airport_code.iloc[a]
    c = row_a['국  가']
    
    for j in range(len(country_code)):
        row_c = country_code.iloc[j]
        if row_c['나라 이름'] == c:
            airport_code.loc[a, '영문 이름'] = row_c['alpha-3']
""")


In [ ]:
### 방법1 : 판다스 iterrows() 함수
measure_run_time("""
for a, row_a in airport_code.iterrows() :
    c = row_a['국  가']
    
    for j, row_c in country_code.iterrows() :
        if row_c['나라 이름'] == c :
            airport_code.loc[a,'영문 이름'] = row_c['alpha-3']
""")


In [ ]:
airport_code.info()

In [ ]:
country_code.head(1)

In [ ]:
airport_code[airport_code['영문 이름'].isnull()]

In [ ]:
airport_code = airport_code.dropna()

In [ ]:
airport_code.isnull().sum()

In [ ]:
airport_code[airport_code['영문 이름'] == '']

In [ ]:
airport_code.head()

In [ ]:
asia.info()

In [ ]:
#asia = asia.dropna(axis=1)

In [ ]:
asia.head()

In [ ]:
asia_ = asia[['출발 국가','도착 국가']].copy()

In [ ]:
asia_

In [ ]:
asia_.shape

In [ ]:
asia_.head()

In [ ]:
asia_depart = asia_.merge(country_code, 
                          left_on="출발 국가", 
                          right_on="alpha-3",
                          sort=False,how='left')

In [ ]:
asia_depart.isnull().sum()

In [ ]:
asia_depart.columns

In [ ]:
asia_depart = asia_depart.rename(columns={'나라 이름': "출발국가이름"})

In [ ]:
asia_depart.head()

In [ ]:
asia_depart = asia_depart.drop(['숫자','alpha-3','alpha-2'], axis=1)

In [ ]:
asia_depart.head()

In [ ]:
asia_arrival = asia_depart.merge(country_code,
                                 left_on="도착 국가", 
                                 right_on="alpha-3",
                                 sort=False,
                                 how='left')

In [ ]:
asia_arrival.shape

In [ ]:
asia_arrival.isnull().sum()

In [ ]:
asia_arrival = asia_arrival.rename(columns={'나라 이름': "도착국가이름"})

In [ ]:
asia_arrival.head()

In [ ]:
asia_arrival = asia_arrival.drop(['숫자','alpha-3','alpha-2'], axis=1)

In [ ]:
asia_arrival.head()

In [ ]:
## 사이즈가 큰 데이터프레임에 대한 merge 작업시 메모리 오버로드

In [ ]:
len(asia)

In [ ]:
len(asia_arrival)

In [ ]:
asia.head(1)

In [ ]:
# try :
#     aisa = asia.merge(asia_arrival, on="출발 국가")
# except MemoryError as e :
#     print('MemoryError', e)

In [ ]:
asia = pd.concat([asia, asia_arrival[['출발국가이름','도착국가이름']]], axis=1)

In [ ]:
asia

In [ ]:
# asia.get_dtype_counts()
# AttributeError: 'DataFrame' object has no attribute 'get_dtype_counts'
asia.dtypes.value_counts()

In [ ]:
asia.head()

In [ ]:
asia.tail()

## 특별 조건별 데이터 분할하기

In [ ]:
#asia.columns
asia.head()

In [ ]:
asia_dep_port = asia.groupby(["항공사","출발 공항","출발 국가","출발국가이름","도착 공항","도착 국가","년도","월"])

In [ ]:
asia_dep_port

In [ ]:
asia_dep_port.head()

In [ ]:
a = asia_dep_port.agg({'월':'count',
                       '년도':'count',
                       '출발 국가':'count', 
                       '도착 국가':'count',
                       '정원(전체)':'sum'}) 

In [ ]:
a.head(20)

In [ ]:
a.columns

In [ ]:
a.columns = ['운항회수', '년도', '출발 국가', '도착 국가', '정원(전체)']

In [ ]:
a.head()

In [ ]:
a.drop(columns="년도",axis=1,inplace=True)

In [ ]:
a.head()

In [ ]:
a.to_csv("./output/example_lee.csv", encoding='euc-kr')

In [ ]:
asia_출발국가  = asia.groupby(["출발 국가", 
                           "항공사",
                           "출발 공항",
                           "도착 공항",
                           "도착 국가",
                           "도착국가이름",
                           "년도",
                           "월"])

In [ ]:
aa = asia_출발국가.agg({'월':'count',
                    '년도':'count',
                    '출발 국가':'count',
                    '도착 국가':'count',
                    '정원(전체)':'sum'}) 

In [ ]:
# aa = asia_출발국가.agg({'정원(전체)':['count','sum']}) 

In [ ]:
aa.head()

In [ ]:
aa.columns = ['운항회수', '년도', '출발 국가', '도착 국가', '정원(전체)']

In [ ]:
aa.head()

In [ ]:
aa.drop(columns="년도",axis=1,inplace=True)

In [ ]:
aa.head(20)

In [ ]:
aa.to_csv("./output/example_lee_dep_con.csv",encoding='euc-kr')

In [ ]:
asia_도착국가  = asia.groupby(["도착 국가",
                           "출발 국가", 
                           "항공사",
                           "출발 공항",
                           "도착 공항",
                           "년도",
                           "월"])

In [ ]:
aaa = asia_도착국가.agg({'월':'count',
                     '년도':'count',
                     '출발 국가':'count',
                     '도착 국가':'count',
                     '정원(전체)':'sum'})

In [ ]:
aaa.columns = ['운항회수', '년도', '출발 국가', '도착 국가', '정원(전체)']

In [ ]:
aaa.drop(columns="년도",axis=1,inplace=True)

In [ ]:
aaa.head()

In [ ]:
aaa.to_csv("./output/example_lee_ari_con.csv",encoding='euc-kr')